In [ ]:
#import basic requirements
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as utils
import numpy as np
dtype= torch.cuda.FloatTensor

In [ ]:
def load_data(path='mnist.npz'):
    """Loads the MNIST dataset.
    # Arguments
        path: path where mnist is stored locallym, download from https://s3.amazonaws.com/img-datasets/mnist.npz
            
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
    return (x_train, y_train), (x_test, y_test)


In [ ]:
def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]

    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    return categorical

In [ ]:
#create dataset for training and testing
batch_size = 128
num_classes = 10
epochs = 20

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
#print(x_train.shape, 'train samples')
#print(x_test.shape, 'test samples')
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)
ypred=[]


In [ ]:
#create a two layer mLP
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1=nn.Linear(784,512)
        self.fc2=nn.Linear(512,50)
        self.fc3=nn.Linear(50,10)
     
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=(self.fc3(x))
        
        return x
    
    

In [ ]:
#convert dataset and train
net=Net()
net=net.cuda()
#convert numpy arrays to torch tensors
Xtr=torch.stack([torch.Tensor(i) for i in x_train])
Ytr=torch.stack([torch.Tensor(i) for i in y_train])

Xte=torch.stack([torch.Tensor(i) for i in x_test])
Yte=torch.stack([torch.Tensor(i) for i in y_test])

traindataset=utils.TensorDataset(Xtr,Ytr)
traindataloader=utils.DataLoader(traindataset)


testdataset=utils.TensorDataset(Xte,Yte)
testdataloader=utils.DataLoader(testdataset)



epochs=100
optimizer=optim.SGD(net.parameters(),lr=0.01)
criterion=torch.nn.MSELoss()

In [ ]:
def train():
    net.train()
    for batch,(data,target) in enumerate(traindataloader):
        data, target = data.cuda(), target.cuda()
        data,target=Variable(data),Variable(target)
        optimizer.zero_grad()
        #print data.size()
        output=net(data)
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
        if batch % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch * len(data), len(traindataloader.dataset),
                100. * batch / len(traindataloader), loss.data[0]))


def test():
    net.eval()
    correct=0
    testloss=0
    
    #todo: must optimize bunch of functions
    for data,target in testdataloader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        output = net(data).cpu().data.numpy()[0]
        output=np.array(output)
        op=[]
        for i in output:
            if i==np.max(output):
                op.append(1)
            else:
                op.append(0)
        ypred.append(op)
        #print op



In [ ]:
#test for multiple epochs
for epoch in range(1):
    train()
    test()
ynewpr=[np.argmax(i,axis=-1) for i in ypred]
ynewte=[np.argmax(i,axis=-1) for i in y_test]
from sklearn.metrics import classification_report
print(classification_report(ynewte, ynewpr))